In [1]:
%matplotlib inline
import numpy as np
from scipy.special import gamma
import nbimporter
from _helper_functions import cauchy, tsallis, find_nearest

Importing Jupyter notebook from _helper_functions.ipynb


In [2]:
def create_spins(gcoll, q, delta, gammaq, nbins, holes=False, inhomo=True, hyperfine=True):
    
    #delta: ws- wc 
    
    hfcoupl = 2.3

    f = np.linspace(-gammaq/2, gammaq/2, nbins )
    if hyperfine:
        pdf = tsallis(f, q, gammaq/2/np.pi) + tsallis(f-hfcoupl, q, gammaq/2/np.pi)+tsallis(f+hfcoupl, q, gammaq/2/np.pi)
    else:
        pdf = tsallis(f, q, gammaq/2/np.pi)

    if holes:
        frequency_exclude = gcoll/2/np.pi
        exclude_width = 0.2 #FWHM

        indf = find_nearest(f, frequency_exclude)
        scale = pdf[indf]

        toex = cauchy(f, frequency_exclude, exclude_width, -scale)+cauchy(f, -frequency_exclude, exclude_width, -scale)
        #toex2 = cauchy(f, 0, exclude_width, -pdf[int(len(f)/2)])
        pdf = pdf+toex

    pdf = np.clip(pdf, 0, max(pdf) )   

    spdf = sum(pdf)
    pdf = pdf/spdf    
    
    f += delta/2/np.pi
    
    if not inhomo:
        pdf = np.zeros(np.shape(pdf))
        pdf[int(nbins/2)] = 1

    spins = f #draw samples according to distribution
    gs = np.sqrt(pdf)*gcoll
    
    return spins, gs, pdf